In [1]:
from collections import defaultdict
from itertools import product
from math import factorial
from timeit import default_timer as timer

from dsrg_generator.helper.file_utils import ChangeDir
from dsrg_generator import space_relation
from dsrg_generator import Term, hamiltonian_operator, cluster_operator
from dsrg_generator import diagonal_fock_operator, fink_hamiltonian, dyall_hamiltonian
from dsrg_generator import contract_terms, combine_terms
from dsrg_generator import nested_commutator_cc, contract_terms
from dsrg_generator import single_commutator, recursive_single_commutator, bch_cc_rsc
from dsrg_generator import linear_commutator, linear_commutator_amp, cluster_operators
from dsrg_generator import categorize_contractions
from dsrg_generator import print_terms_ambit
from dsrg_generator.write_forte_functions import save_terms_ambit_functions, save_direct_t3

In [2]:
h0 = combine_terms(diagonal_fock_operator(False))
h1 = combine_terms([x.canonicalize() for i in [1,2] for x in hamiltonian_operator(i).expand_composite_indices(False)] +
                   [Term.from_term(i, True).canonicalize() for i in h0])

In [3]:
h0

[1 H^{ v_{1} }_{ v_{0} } a^{ v_{1} }_{ v_{0} },
 1 H^{ c_{1} }_{ c_{0} } a^{ c_{1} }_{ c_{0} },
 1 H^{ a_{1} }_{ a_{0} } a^{ a_{1} }_{ a_{0} }]

In [4]:
h1

[1 H^{ c_{0} }_{ v_{0} } a^{ v_{0} }_{ c_{0} },
 1 H^{ a_{0} }_{ v_{0} } a^{ v_{0} }_{ a_{0} },
 1 H^{ c_{0} }_{ v_{0} } a^{ c_{0} }_{ v_{0} },
 1 H^{ a_{0} }_{ c_{0} } a^{ c_{0} }_{ a_{0} },
 1 H^{ a_{0} }_{ v_{0} } a^{ a_{0} }_{ v_{0} },
 1 H^{ a_{0} }_{ c_{0} } a^{ a_{0} }_{ c_{0} },
 1/4 H^{ c_{0} c_{1} }_{ v_{0} v_{1} } a^{ v_{0} v_{1} }_{ c_{0} c_{1} },
 1/2 H^{ c_{0} a_{0} }_{ v_{0} v_{1} } a^{ v_{0} v_{1} }_{ c_{0} a_{0} },
 1/4 H^{ a_{0} a_{1} }_{ v_{0} v_{1} } a^{ v_{0} v_{1} }_{ a_{0} a_{1} },
 1/2 H^{ a_{0} a_{1} }_{ v_{0} c_{0} } a^{ v_{0} c_{0} }_{ a_{0} a_{1} },
 1 H^{ c_{0} a_{0} }_{ v_{0} c_{1} } a^{ v_{0} c_{1} }_{ c_{0} a_{0} },
 1/2 H^{ c_{0} c_{1} }_{ v_{0} c_{2} } a^{ v_{0} c_{2} }_{ c_{0} c_{1} },
 1/2 H^{ c_{0} c_{1} }_{ v_{0} a_{0} } a^{ v_{0} a_{0} }_{ c_{0} c_{1} },
 1 H^{ c_{0} a_{0} }_{ v_{0} a_{1} } a^{ v_{0} a_{1} }_{ c_{0} a_{0} },
 1/2 H^{ a_{0} a_{1} }_{ v_{0} a_{2} } a^{ v_{0} a_{2} }_{ a_{0} a_{1} },
 1/2 H^{ v_{1} v_{2} }_{ v_{0} c_{0} } a^{ v_{1} v

In [5]:
h = {i: combine_terms([x for x in hamiltonian_operator(i).expand_composite_indices(False)]) for i in [1, 2]}
t = {i: [j for j in cluster_operator(i, excitation=True, start=20).expand_composite_indices(False, 20)]
     for i in [1, 2]}
td = {i: [j for j in cluster_operator(i, excitation=False, scale_factor=-1.0).expand_composite_indices(False, 20)]
      for i in [1, 2]}

t_filter = {1: [], 2: []}
for k, terms in t.items():
    for term in terms:
        if not all([i.space == 'a' for i in term.sq_op.indices()]):
            t_filter[k].append(term)

td_filter = {1: [], 2: []}
for k, terms in td.items():
    for term in terms:
        if not all([i.space == 'a' for i in term.sq_op.indices()]):
            td_filter[k].append(term)

a = {k: [Term.from_term(i) for i in v] for k, v in t_filter.items()}
for i in [1, 2]:
    a[i] += [Term.from_term(term) for term in td_filter[i]]

# h3ddca = combine_terms([i for term in h[3] for i in term.make_ddca(4, 4, True)])
# h1d = [term for term in h[1] if len(set(i.space for i in term.indices_set)) == 1]
# h1od = [term for term in h[1] if len(set(i.space for i in term.indices_set)) != 1]

In [6]:
# print_terms_ambit(a[1])
# print_terms_ambit(a[2])
a

{1: [1 T^{ c_{20} }_{ a_{20} } a^{ a_{20} }_{ c_{20} },
  1 T^{ c_{20} }_{ v_{20} } a^{ v_{20} }_{ c_{20} },
  1 T^{ a_{20} }_{ v_{20} } a^{ v_{20} }_{ a_{20} },
  -1 T^{ c_{20} }_{ a_{20} } a^{ c_{20} }_{ a_{20} },
  -1 T^{ c_{20} }_{ v_{20} } a^{ c_{20} }_{ v_{20} },
  -1 T^{ a_{20} }_{ v_{20} } a^{ a_{20} }_{ v_{20} }],
 2: [1/4 T^{ c_{20} c_{21} }_{ a_{20} a_{21} } a^{ a_{20} a_{21} }_{ c_{20} c_{21} },
  1/4 T^{ c_{20} a_{22} }_{ a_{20} a_{21} } a^{ a_{20} a_{21} }_{ c_{20} a_{22} },
  1/4 T^{ a_{22} c_{20} }_{ a_{20} a_{21} } a^{ a_{20} a_{21} }_{ a_{22} c_{20} },
  1/4 T^{ c_{20} c_{21} }_{ a_{20} v_{20} } a^{ a_{20} v_{20} }_{ c_{20} c_{21} },
  1/4 T^{ c_{20} a_{21} }_{ a_{20} v_{20} } a^{ a_{20} v_{20} }_{ c_{20} a_{21} },
  1/4 T^{ a_{21} c_{20} }_{ a_{20} v_{20} } a^{ a_{20} v_{20} }_{ a_{21} c_{20} },
  1/4 T^{ a_{21} a_{22} }_{ a_{20} v_{20} } a^{ a_{20} v_{20} }_{ a_{21} a_{22} },
  1/4 T^{ c_{20} c_{21} }_{ v_{20} a_{20} } a^{ v_{20} a_{20} }_{ c_{20} c_{21} },
  1/4 T^

In [11]:
H0A = linear_commutator(h0, a[2], 1.0, 0, 4)
H0A

Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 1.
1 1
in func 1
Max cumulant level is set to 1.
Max cumulant level is set to 1.
1 1
in func 1
Max cumulant level is set to 1.
Max cumulant level is set to 1.
1 1
in func 1
Max cumulant level is set to 1.
Max cumulant level is set to 1.
1 1
in func 1
Max cumulant level is set to 1.
Max cumulant level is set to 1.
1 1
in func 1
Max cumulant level is set to 1.
Max cumulant level is set to 1.
1 1
in func 1
Max cumulant level is set to 1.
Max cumulant level is set to 1.
1 1
in func 1
Max cumulant level is set to 1.
Max cumulant level is set to 1.
1 1
in func 1
Max cumulant level is set to 1.
Max cumulant level is set to 1.
2 2
in func 2
Max cumulant level is set to 1.
Max cumulant level is set to 1.
2 2
in func 2
Max cumulant level is set to 1.
Max cumulant level is set to

[-1 H^{ a_{1} }_{ a_{0} } T^{ c_{0} a_{0} }_{ v_{0} a_{2} } L^{ a_{2} }_{ a_{1} } a^{ v_{0} }_{ c_{0} },
 1 H^{ a_{1} }_{ a_{0} } T^{ c_{0} a_{2} }_{ v_{0} a_{0} } L^{ a_{2} }_{ a_{1} } a^{ v_{0} }_{ c_{0} },
 -1 H^{ a_{2} }_{ a_{1} } T^{ a_{0} a_{1} }_{ v_{0} a_{3} } L^{ a_{3} }_{ a_{2} } a^{ v_{0} }_{ a_{0} },
 1 H^{ a_{2} }_{ a_{1} } T^{ a_{0} a_{3} }_{ v_{0} a_{1} } L^{ a_{3} }_{ a_{2} } a^{ v_{0} }_{ a_{0} },
 -1 H^{ a_{1} }_{ a_{0} } T^{ c_{0} a_{0} }_{ v_{0} a_{2} } L^{ a_{2} }_{ a_{1} } a^{ c_{0} }_{ v_{0} },
 1 H^{ a_{1} }_{ a_{0} } T^{ c_{0} a_{2} }_{ v_{0} a_{0} } L^{ a_{2} }_{ a_{1} } a^{ c_{0} }_{ v_{0} },
 1 H^{ a_{2} }_{ a_{1} } T^{ a_{0} a_{1} }_{ c_{0} a_{3} } L^{ a_{3} }_{ a_{2} } a^{ c_{0} }_{ a_{0} },
 -1 H^{ a_{2} }_{ a_{1} } T^{ a_{0} a_{3} }_{ c_{0} a_{1} } L^{ a_{3} }_{ a_{2} } a^{ c_{0} }_{ a_{0} },
 -1 H^{ a_{2} }_{ a_{1} } T^{ a_{0} a_{1} }_{ v_{0} a_{3} } L^{ a_{3} }_{ a_{2} } a^{ a_{0} }_{ v_{0} },
 1 H^{ a_{2} }_{ a_{1} } T^{ a_{0} a_{3} }_{ v_{0} a_{1} } 

In [12]:
H0AA = linear_commutator(H0A, a[2], 0.5, 0, 0)
H0AA

Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumu

Max cumulant level is set to 2.
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 2.
Max cumulant level is set to 2.
1 1
in func 1
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 2.
Max cumulant level is set to 2.
1 1
in func 1
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant

Max cumulant level is set to 1.
Max cumulant level is set to 1.
0 0
Max cumulant level is set to 2.
Max cumulant level is set to 2.
Max cumulant level is set to 2.
Max cumulant level is set to 2.
Max cumulant level is set to 2.
Max cumulant level is set to 2.
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max 

0 0
Max cumulant level is set to 1.
Max cumulant level is set to 1.
0 0
Max cumulant level is set to 1.
Max cumulant level is set to 1.
0 0
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 2.
0 0
0 0
Max cumulant level is set to 2.
0 0
0 0
Max cumulant level is set to 2.
0 0
0 0
Max cumulant level is set to 2.
0 0
0 0
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 2.
0 0
0 0
Max cumulant level is set to 2.
0 0
0 0
Max cumulant level is set to 2.
0 0
0 0
Max cumulant level is set to 2.
0 0
0 0
Max cumulant level is set to 2.
0 0
0 0
Max cumulant level is set to 2.
0 0
0 0
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 2.
0 0
0 0
Max cumulant level is set to 2.
0 0


0 0
0 0
Max cumulant level is set to 2.
0 0
0 0
Max cumulant level is set to 2.
0 0
0 0
Max cumulant level is set to 2.
Max cumulant level is set to 2.
0 0
0 0
0 0
0 0
0 0
4 4
8 8
10 10
11 11
in func 11
0 0
0 0
0 0
0 0
4 4
8 8
10 10
11 11
in func 11
Max cumulant level is set to 2.
Max cumulant level is set to 2.
0 0
0 0
Max cumulant level is set to 2.
Max cumulant level is set to 2.
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
Max cumulant level is set to 2.
Max cumulant level is set to 2.
Max cumulant level is set to 2.
0 0
0 0
Max cumulant level is set to 2.
0 0
0 0
Max cumulant level is set to 2.
0 0
0 0
Max cumulant level is set to 2.
0 0
0 0
Max cumulant level is set to 2.
0 0
0 0
Max cumulant level is set to 2.
0 0
0 0
Max cumulant level is set to 2.
Max cumulant level is set to 2.
Max cumulant level is set to 2.
0 0
0 0
Max cumulant level is set to 2.
0 0
0 0
Max cumulant level is set to 2.
0 0
0 0
Max cumulant level is set to 2.
0 0
0 0
Max cumulant level is 

...

0 0
0 0
Max cumulant level is set to 2.
0 0
0 0
Max cumulant level is set to 2.
0 0
0 0
Max cumulant level is set to 2.
0 0
0 0
Max cumulant level is set to 2.
0 0
0 0
Max cumulant level is set to 2.
0 0
0 0
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 2.
0 0
0 0
Max cumulant level is set to 2.
0 0
0 0
Max cumulant level is set to 2.
0 0
0 0
Max cumulant level is set to 2.
0 0
0 0
Max cumulant level is set to 2.
0 0
0 0
Max cumulant level is set to 2.
0 0
0 0
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 2.
0 0
0 0
Max cumulant level is set to 2.
0 0
0 0
Max cumulant level is set to 2.
0 0
0 0
Max cumulant level is set to 2.
0 0
0 0
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 1.
0 0
Max cumulant level is set to 1.
Max cumulant level is set to 2.
0 0
0 0
Max cumulant level is set to 2.
0 0
0 0
Max cumulant level is set to 2.
0 0
0 0
Max cumulant

Max cumulant level is set to 2.
0 0
0 0
Max cumulant level is set to 2.
0 0
0 0
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 2.
0 0
0 0
Max cumulant level is set to 2.
0 0
0 0
Max cumulant level is set to 2.
0 0
0 0
Max cumulant level is set to 2.
0 0
0 0
Max cumulant level is set to 2.
0 0
0 0
Max cumulant level is set to 2.
0 0
0 0
Max cumulant level is set to 2.
Max cumulant level is set to 2.
Max cumulant level is set to 2.
0 0
0 0
Max cumulant level is set to 2.
0 0
0 0
Max cumulant level is se

Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 2.
Max cumulant level is set to 2.
Max cumulant level is set to 2.
Max cumulant level is set to 2.
16 16
18 18
in func 18
Max cumulant level is set to 1.
Max cumulant level is set to 1.
0 0
Max cumulant level is set to 1.
Max cumulant level is set to 1.
0 0
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 2.
Max cumulant level is set to 2.
0 0
0 0
Max cumulant level is set to 1.
Max cumulant level is set to 1.
0 0
Max cumulant level is set to 1.
Max cumulant level is set to 1.
0 0
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 2.
Max cumulant level is set to 2.
0 0
0 0
Max cumulant level is set to 1.
Max cumulant level is set to 1.
0 0
Max cumulant level is set to 1.
Max cumulant level is set to 1.
0 0
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumul

17 17
18 18
in func 18
Max cumulant level is set to 2.
0 0
0 0
Max cumulant level is set to 2.
0 0
0 0
Max cumulant level is set to 2.
0 0
0 0
Max cumulant level is set to 2.
0 0
0 0
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 2.
0 0
0 0
Max cumulant level is set to 2.
0 0
0 0
Max cumulant level is set to 2.
0 0
0 0
Max cumulant level is set to 2.
0 0
0 0
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 2.
0 0
0 0
Max cumulant level is set to 2.
0 0
0 0
Max cumulant level is set to 2.
0 0
0 0
Max cumulant level is set to 2.
0 0
0 0
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 2.
0 0
0 0
Max cumulant level is set to 2.
0 0
0 0
Max cumulant level is set to 2.
0 0
0 0
Max cumulant level is set to 2.
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
Max cumulant level is set to 1.
Max cumulant leve

[1/2 H^{ v_{1} }_{ v_{0} } T^{ c_{0} c_{1} }_{ v_{0} v_{2} } T^{ c_{0} c_{1} }_{ v_{1} v_{2} } a^{  }_{  },
 1/2 H^{ v_{1} }_{ v_{0} } T^{ c_{0} c_{1} }_{ v_{0} a_{0} } T^{ c_{0} c_{1} }_{ v_{1} a_{0} } a^{  }_{  },
 -1/2 H^{ c_{1} }_{ c_{0} } T^{ c_{0} c_{2} }_{ v_{0} v_{1} } T^{ c_{1} c_{2} }_{ v_{0} v_{1} } a^{  }_{  },
 -1 H^{ c_{1} }_{ c_{0} } T^{ c_{0} c_{2} }_{ v_{0} a_{0} } T^{ c_{1} c_{2} }_{ v_{0} a_{0} } a^{  }_{  },
 -1/2 H^{ c_{1} }_{ c_{0} } T^{ a_{0} a_{1} }_{ c_{0} c_{2} } T^{ a_{0} a_{1} }_{ c_{1} c_{2} } a^{  }_{  },
 1/2 H^{ a_{1} }_{ a_{0} } T^{ c_{0} c_{1} }_{ v_{0} a_{0} } T^{ c_{0} c_{1} }_{ v_{0} a_{1} } a^{  }_{  },
 1/2 H^{ a_{1} }_{ a_{0} } T^{ a_{0} a_{2} }_{ c_{0} c_{1} } T^{ a_{1} a_{2} }_{ c_{0} c_{1} } a^{  }_{  },
 -1/2 H^{ v_{1} }_{ v_{0} } T^{ c_{0} c_{1} }_{ v_{0} a_{0} } T^{ c_{0} c_{1} }_{ v_{1} a_{1} } L^{ a_{1} }_{ a_{0} } a^{  }_{  },
 1 H^{ v_{1} }_{ v_{0} } T^{ c_{0} a_{0} }_{ v_{0} v_{2} } T^{ c_{0} a_{1} }_{ v_{1} v_{2} } L^{ a_{1} }_{ a_{0}

In [13]:
HA = linear_commutator(h[1] + h[2], a[2], 1.0, 0, 0)
HA

Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumu

Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumu

Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 1.
0 0
Max cumulant level is set to 1.
Max cumulant level is set to 1.
0 0
Max cumulant level is set to 1.
Max cumulant level is set to 1.
0 0
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 1.
0 0
Max cumulant level is set to 1.
Max cumulant level is set to 1.
0 0
Max cumulant level is set to 1.
Max cumulant level is set to 1.
0 0
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 1.
0 0
Max cumulant level is set to 1.
Max cumulant level is set to 1.
0 0
Max cumulant level is set to 1.
Max cumulant level is set to 1.
0 0
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max 

Max cumulant level is set to 1.
0 0
Max cumulant level is set to 1.
Max cumulant level is set to 1.
0 0
Max cumulant level is set to 1.
Max cumulant level is set to 1.
0 0
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 1.
0 0
Max cumulant level is set to 1.
Max cumulant level is set to 1.
0 0
Max cumulant level is set to 1.
Max cumulant level is set to 1.
0 0
Max cumulant level is set to 1.
Max cumulant level is set to 1.
0 0
Max cumulant level is set to 2.
Max cumulant level is set to 2.
0 0
Max cumulant level is set to 1.
Max cumulant level is set to 1.
0 0
Max cumulant level is set to 1.
Max cumulant level is set to 1.
0 0
Max cumulant level is set to 1.
Max cumulant level is set to 1.
0 0
Max cumulant level is set to 2.
Max cumulant level is set to 2.
0 0
Max cumulant level is set to 1.
Max cumulant level is set to 1.
0 0
Max cumulant level is set to 1.
Max cumulant level is set to 1.
0 0
Max cumulant level is set to 1.
Max cumulant lev

in func 3
Max cumulant level is set to 1.
Max cumulant level is set to 1.
0 0
Max cumulant level is set to 2.
0 0
0 0
Max cumulant level is set to 2.
0 0
0 0
Max cumulant level is set to 2.
0 0
0 0
Max cumulant level is set to 2.
2 2
3 3
in func 3
Max cumulant level is set to 2.
0 0
0 0
Max cumulant level is set to 2.
2 2
3 3
in func 3
Max cumulant level is set to 1.
Max cumulant level is set to 1.
0 0
Max cumulant level is set to 2.
0 0
0 0
Max cumulant level is set to 2.
0 0
0 0
Max cumulant level is set to 2.
0 0
0 0
Max cumulant level is set to 2.
0 0
0 0
Max cumulant level is set to 1.
Max cumulant level is set to 1.
0 0
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 2.
0 0
0 0
Max cumulant level is set to 2.
0 0
0 0
Max cumulant level is set to 2.
0 0
0 0
Max cumulant level is set to 2.
0 0
0 0
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 2.
0 0
0 0
Max cumulant level is set to 2.
0 0
0 

...

Max cumulant level is set to 2.
Max cumulant level is set to 2.
0 0
0 0
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 1.
0 0
Max cumulant level is set to 1.
Max cumulant level is set to 1.
0 0
Max cumulant level is set to 1.
Max cumulant level is set to 1.
0 0
Max cumulant level is set to 1.
Max cumulant level is set to 2.
0 0
Max cumulant level is set to 2.
Max cumulant level is set to 1.
0 0
Max cumulant level is set to 1.
Max cumulant level is set to 1.
0 0
Max cumulant level is set to 1.
Max cumulant level is set to 1.
0 0
Max cumulant level is set to 1.
Max cumulant level is set to 2.
0 0
Max cumulant level is set to 2.
Max cumulant level is set to 1.
0 0
Max cumulant level is set to 1.
Max cumulant level is set to 1.
2 2
in func 2
Max cumulant level is set to 1.
Max cumulant level is set to 1.
2 2
in func 2


0 0
0 0
Max cumulant level is set to 2.
0 0
0 0
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 1.
0 0
Max cumulant level is set to 1.
Max cumulant level is set to 2.
0 0
0 0
Max cumulant level is set to 2.
0 0
0 0
Max cumulant level is set to 2.
0 0
0 0
Max cumulant level is set to 2.
0 0
0 0
Max cumulant level is set to 1.
0 0
Max cumulant level is set to 1.
Max cumulant level is set to 2.
2 2
3 3
in func 3
Max cumulant level is set to 2.
0 0
0 0
Max cumulant level is set to 2.
2 2
3 3
in func 3
Max cumulant level is set to 2.
0 0
0 0
Max cumulant level is set to 2.
0 0
0 0
Max cumulant level is set to 2.
0 0
0 0
Max cumulant level is set to 1.
0 0
Max cumulant level is set to 1.
Max cumulant level is set to 2.
2 2
3 3
in func 3
Max cumulant level is set to 2.
0 0
0 0
Max cumulant level is set to 2.
2 2
3 3
in func 3
Max cumulant level is set to 2.
0 0
0 0
Max cumulant level is set to 2.
0 0
0 0
Max cumulant level is set to 2.
0 0
0 0
Max 

0 0
0 0
0 0
0 0
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 2.
0 0
0 0
Max cumulant level is set to 2.
0 0
0 0
Max cumulant level is set to 2.
0 0
0 0
Max cumulant level is set to 2.
0 0
0 0
4 4
8 8
10 10
11 11
in func 11
0 0
0 0
0 0
0 0
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 2.
0 0
Max cumulant level is set to 2.
Max cumulant level is set to 2.
0 0
Max cumulant level is set to 2.
Max cumulant level is set to 2.
0 0
0 0
Max cumulant level is set to 2.
Max cumulant level is set to 2.
Max cumulant level is set to 2.
Max cumulant level is set to 2.
Max cumulant level is set to 2.
Max cumulant level is set to 2.
Max cumulant level is set to 2.
Max cumulant level is set to 1.
Max cumulant level is set to 1.
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
Max cumulant level is set to 2.
0 0
0 0
Max cumulant level is set to 2.
0 0
0 0
Max cumulant level is set to 2.
0 0
0 0
Max cumulant level is set to 2.
0

Max cumulant level is set to 2.
0 0
0 0
Max cumulant level is set to 2.
0 0
0 0
Max cumulant level is set to 2.
0 0
0 0
Max cumulant level is set to 2.
0 0
0 0
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 2.
0 0
0 0
Max cumulant level is set to 2.
0 0
0 0
Max cumulant level is set to 2.
0 0
0 0
Max cumulant level is set to 2.
0 0
0 0
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 2.
Max cumulant level is set to 2.
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
Max cumulant level is set to 2.
Max cumulant level is set to 2.
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
Max cumulant level is set to 2.
Max cumulant level is set to 2.
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
Max cumulant level is set to 2.
Max cumulant level is set to 2.
Max cumulant level is set to 2.
Max cumulant level is set to 2.
Max cumulant level is set to 2.
Max cumu

[1/2 H^{ c_{0} c_{1} }_{ v_{0} v_{1} } T^{ c_{0} c_{1} }_{ v_{0} v_{1} } a^{  }_{  },
 1 H^{ c_{0} c_{1} }_{ v_{0} a_{0} } T^{ c_{0} c_{1} }_{ v_{0} a_{0} } a^{  }_{  },
 1/2 H^{ a_{0} a_{1} }_{ c_{0} c_{1} } T^{ a_{0} a_{1} }_{ c_{0} c_{1} } a^{  }_{  },
 1 H^{ a_{0} }_{ v_{0} } T^{ a_{2} a_{3} }_{ v_{0} a_{1} } L^{ a_{2} a_{3} }_{ a_{0} a_{1} } a^{  }_{  },
 -1 H^{ a_{0} }_{ c_{0} } T^{ a_{2} a_{3} }_{ c_{0} a_{1} } L^{ a_{2} a_{3} }_{ a_{0} a_{1} } a^{  }_{  },
 -1 H^{ c_{0} c_{1} }_{ v_{0} a_{0} } T^{ c_{0} c_{1} }_{ v_{0} a_{1} } L^{ a_{1} }_{ a_{0} } a^{  }_{  },
 1 H^{ c_{0} a_{0} }_{ v_{0} v_{1} } T^{ c_{0} a_{1} }_{ v_{0} v_{1} } L^{ a_{1} }_{ a_{0} } a^{  }_{  },
 2 H^{ c_{0} a_{1} }_{ v_{0} a_{0} } T^{ c_{0} a_{2} }_{ v_{0} a_{0} } L^{ a_{2} }_{ a_{1} } a^{  }_{  },
 2 H^{ c_{0} a_{1} }_{ v_{0} a_{0} } T^{ c_{0} a_{3} }_{ v_{0} a_{2} } L^{ a_{1} a_{2} }_{ a_{0} a_{3} } a^{  }_{  },
 1/4 H^{ a_{0} a_{1} }_{ v_{0} v_{1} } T^{ a_{2} a_{3} }_{ v_{0} v_{1} } L^{ a_{2} a_{3} }_{ a

In [14]:
print_terms_ambit(H0AA)

C0 += (1.0 / 2.0) * H1["v1,v0"] * T2["c0,c1,v0,v2"] * T2["c0,c1,v1,v2"];
C0 += (1.0 / 2.0) * H1["v1,v0"] * T2["c0,c1,v0,a0"] * T2["c0,c1,v1,a0"];
C0 += (-1.0 / 2.0) * H1["c1,c0"] * T2["c0,c2,v0,v1"] * T2["c1,c2,v0,v1"];
C0 += -1.0 * H1["c1,c0"] * T2["c0,c2,v0,a0"] * T2["c1,c2,v0,a0"];
C0 += (-1.0 / 2.0) * H1["c1,c0"] * T2["c0,c2,a0,a1"] * T2["c1,c2,a0,a1"];
C0 += (1.0 / 2.0) * H1["a1,a0"] * T2["c0,c1,v0,a0"] * T2["c0,c1,v0,a1"];
C0 += (1.0 / 2.0) * H1["a1,a0"] * T2["c0,c1,a0,a2"] * T2["c0,c1,a1,a2"];
C0 += (-1.0 / 2.0) * H1["v1,v0"] * T2["c0,c1,v0,a0"] * T2["c0,c1,v1,a1"] * L1["a1,a0"];
C0 += 1.0 * H1["v1,v0"] * T2["c0,a0,v0,v2"] * T2["c0,a1,v1,v2"] * L1["a1,a0"];
C0 += 1.0 * H1["v1,v0"] * T2["c0,a1,v0,a0"] * T2["c0,a2,v1,a0"] * L1["a2,a1"];
C0 += 1.0 * H1["v1,v0"] * T2["c0,a1,v0,a0"] * T2["c0,a3,v1,a2"] * L2["a1,a2,a0,a3"];
C0 += (1.0 / 4.0) * H1["v1,v0"] * T2["a0,a1,v0,v2"] * T2["a2,a3,v1,v2"] * L2["a2,a3,a0,a1"];
C0 += (1.0 / 4.0) * H1["v1,v0"] * T2["a1,a2,v0,a0"] * T2["a3,a4,v1,a0"

In [10]:
energy = combine_terms([Term.from_term(term) for term in HA + H0AA])
print_terms_ambit(energy)

C0 += 2.0 * H1["c0,v0"] * T1["c0,v0"];
C0 += 2.0 * H1["a0,c0"] * T1["c0,a0"];
C0 += (1.0 / 2.0) * H2["c0,c1,v0,v1"] * T2["c0,c1,v0,v1"];
C0 += 1.0 * H2["c0,c1,v0,a0"] * T2["c0,c1,v0,a0"];
C0 += (1.0 / 2.0) * H2["a0,a1,c0,c1"] * T2["c0,c1,a0,a1"];
C0 += 1.0 * H1["v1,v0"] * T1["c0,v0"] * T1["c0,v1"];
C0 += (1.0 / 2.0) * H1["v1,v0"] * T2["c0,c1,v0,v2"] * T2["c0,c1,v1,v2"];
C0 += (1.0 / 2.0) * H1["v1,v0"] * T2["c0,c1,v0,a0"] * T2["c0,c1,v1,a0"];
C0 += -1.0 * H1["c1,c0"] * T1["c0,v0"] * T1["c1,v0"];
C0 += -1.0 * H1["c1,c0"] * T1["c0,a0"] * T1["c1,a0"];
C0 += (-1.0 / 2.0) * H1["c1,c0"] * T2["c0,c2,v0,v1"] * T2["c1,c2,v0,v1"];
C0 += -1.0 * H1["c1,c0"] * T2["c0,c2,v0,a0"] * T2["c1,c2,v0,a0"];
C0 += (-1.0 / 2.0) * H1["c1,c0"] * T2["c0,c2,a0,a1"] * T2["c1,c2,a0,a1"];
C0 += 2.0 * H1["a0,v0"] * T1["a1,v0"] * L1["a1,a0"];
C0 += 1.0 * H1["a0,v0"] * T2["a2,a3,v0,a1"] * L2["a2,a3,a0,a1"];
C0 += -2.0 * H1["a0,c0"] * T1["c0,a1"] * L1["a1,a0"];
C0 += -1.0 * H1["a0,c0"] * T2["c0,a1,a2,a3"] * L2["a2,a3,a0,

In [11]:
eq_lambda = []

for term in energy:
    for i in term.gradient_t():
        eq_lambda.append(i)

eq_lambda_combined = combine_terms([i for i in eq_lambda])
# for i in eq_lambda_combined:
#     print(i)

In [12]:
print_terms_ambit(eq_lambda_combined)

C1["c0,v0"] += 2.0 * H1["c0,v0"];
C1["c0,v0"] += 2.0 * H1["v1,v0"] * T1["c0,v1"];
C1["c0,v0"] += -2.0 * H1["c1,c0"] * T1["c1,v0"];
C1["c0,v0"] += -1.0 * H1["a1,a0"] * T2["c0,a0,v0,a2"] * L1["a2,a1"];
C1["c0,v0"] += 1.0 * H1["a1,a0"] * T2["c0,a2,v0,a0"] * L1["a2,a1"];

C1["a0,v0"] += 2.0 * H1["a1,v0"] * L1["a1,a0"];
C1["a0,v0"] += 1.0 * H2["a2,a3,v0,a1"] * L2["a2,a3,a0,a1"];
C1["a0,v0"] += 2.0 * H1["v1,v0"] * T1["a1,v1"] * L1["a1,a0"];
C1["a0,v0"] += 1.0 * H1["v1,v0"] * T2["a2,a3,v1,a1"] * L2["a2,a3,a0,a1"];
C1["a0,v0"] += -1.0 * H1["a1,a0"] * T1["a2,v0"] * L1["a2,a1"];
C1["a0,v0"] += (-1.0 / 2.0) * H1["a1,a0"] * T2["a3,a4,v0,a2"] * L2["a3,a4,a1,a2"];
C1["a0,v0"] += -1.0 * H1["a2,a1"] * T1["a1,v0"] * L1["a2,a0"];
C1["a0,v0"] += -1.0 * H1["a2,a1"] * T2["a1,a4,v0,a3"] * L2["a2,a4,a0,a3"];
C1["a0,v0"] += (1.0 / 2.0) * H1["a2,a1"] * T2["a3,a4,v0,a1"] * L2["a3,a4,a0,a2"];
C1["a0,v0"] += 1.0 * H1["a2,a1"] * T2["a1,a4,v0,a3"] * L1["a3,a2"] * L1["a4,a0"];
C1["a0,v0"] += 1.0 * H1["a2,a1"] * T2["

## Dyall Hamiltonian

In [14]:
h0 = combine_terms(dyall_hamiltonian())
h0

[1 H^{ v_{1} }_{ v_{0} } a^{ v_{1} }_{ v_{0} },
 1 H^{ c_{1} }_{ c_{0} } a^{ c_{1} }_{ c_{0} },
 1 H^{ a_{1} }_{ a_{0} } a^{ a_{1} }_{ a_{0} },
 1/4 H^{ a_{2} a_{3} }_{ a_{0} a_{1} } a^{ a_{0} a_{1} }_{ a_{2} a_{3} }]

In [15]:
H0A = linear_commutator(h0, a[1] + a[2], 1.0, 0, 4)
H0A

Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 1.
1 1
in func 1
Max cumulant level is set to 1.
Max cumulant level is set to 1.
1 1
in func 1
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 1.
1 1
in func 1
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 1.
1 1
in func 1
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 1.
1 1
in func 1
Max cumulant level is set to 1.
Max cumulant level is set to 1.
1 1
in func 1
Max cumulant level is set to 1.
Max cumulant level is set to 1.
1 1
in func 1
Max cumulant level is set to 1.
Max cumulant level is set to 1.
1 1
in func 1
Max cumulant level is set to 1.
Max cumulant level is set to 1.
1 1
in func 1
Max cumula

Max cumulant level is set to 1.
2 2
in func 2
Max cumulant level is set to 1.
1 1
in func 1
Max cumulant level is set to 1.
2 2
in func 2
Max cumulant level is set to 1.
4 4
in func 4
Max cumulant level is set to 1.
1 1
in func 1
Max cumulant level is set to 1.
2 2
in func 2
Max cumulant level is set to 1.
1 1
in func 1
Max cumulant level is set to 1.
2 2
in func 2
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 2.
4 4
in func 4
Max cumulant level is set to 2.
2 2
in func 2
Max cumulant level is set to 2.
2 2
in func 2
Max cumulant level is set to 2.
4 4
in func 4
Max cumulant level is set to 2.
Max cumulant level is set to 2.
Max cumulant level is set to 2.
4 4
in func 4
Max cumulant level is set to 2.
2 2
in func 2
Max cumulant level is set to 2.
2 2
in func 2
Max cumulant level is set to 2.
4 4
in func 4
Max cumulant level is set to 2.
Max cumulant level is set to 2.
0 0
0 0
0 0
0 0
16 16
16 16
32 32
36 36
36 36
38 38
38 38
38 38
46 46
54

[1 H^{ v_{1} }_{ v_{0} } T^{ c_{0} }_{ v_{1} } a^{ v_{0} }_{ c_{0} },
 -1 H^{ c_{1} }_{ c_{0} } T^{ c_{1} }_{ v_{0} } a^{ v_{0} }_{ c_{0} },
 -1 H^{ a_{1} }_{ a_{0} } T^{ c_{0} a_{0} }_{ v_{0} a_{2} } L^{ a_{2} }_{ a_{1} } a^{ v_{0} }_{ c_{0} },
 1 H^{ a_{1} }_{ a_{0} } T^{ c_{0} a_{2} }_{ v_{0} a_{0} } L^{ a_{2} }_{ a_{1} } a^{ v_{0} }_{ c_{0} },
 1/2 H^{ a_{2} a_{3} }_{ a_{0} a_{1} } T^{ c_{0} a_{0} }_{ v_{0} a_{4} } L^{ a_{2} a_{3} }_{ a_{1} a_{4} } a^{ v_{0} }_{ c_{0} },
 -1/2 H^{ a_{2} a_{3} }_{ a_{0} a_{1} } T^{ c_{0} a_{4} }_{ v_{0} a_{0} } L^{ a_{2} a_{3} }_{ a_{1} a_{4} } a^{ v_{0} }_{ c_{0} },
 1 H^{ v_{1} }_{ v_{0} } T^{ a_{0} }_{ v_{1} } a^{ v_{0} }_{ a_{0} },
 -1 H^{ a_{1} }_{ a_{0} } T^{ a_{1} }_{ v_{0} } a^{ v_{0} }_{ a_{0} },
 -1 H^{ a_{2} }_{ a_{1} } T^{ a_{0} a_{1} }_{ v_{0} a_{3} } L^{ a_{3} }_{ a_{2} } a^{ v_{0} }_{ a_{0} },
 1 H^{ a_{2} }_{ a_{1} } T^{ a_{0} a_{3} }_{ v_{0} a_{1} } L^{ a_{3} }_{ a_{2} } a^{ v_{0} }_{ a_{0} },
 -1/2 H^{ a_{2} a_{3} }_{ a_{0} a_{1} }

In [16]:
H0AA = linear_commutator(H0A, a[1] + a[2], 0.5, 0, 0)
H0AA

Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 1.
1 1
in func 1
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set 

Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumu

Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 1.
1 1
in func 1
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set 

Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 1.
1 1
in func 1
Max cumulant level is set to 1.
Max cumulant level is set 

Max cumulant level is set to 2.
Max cumulant level is set to 2.
1 1
in func 1
Max cumulant level is set to 2.
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 2.
Max cumulant level is set 

...

0 0
0 0
0 0
0 0
Max cumulant level is set to 2.
0 0
0 0
Max cumulant level is set to 2.
0 0
0 0
Max cumulant level is set to 2.
0 0
0 0
Max cumulant level is set to 2.
0 0
0 0
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 2.
Max cumulant level is set to 2.
Max cumulant level is set to 2.
Max cumulant level is set to 2.
Max cumulant level is set to 2.
Max cumulant level is set to 2.
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 2.
1 1
in func 1
Max cumulant level is set to 2.
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 2.
Max cumulant level is set to 2.
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 2.
0 0
0 0
Max cumulant level is set to 2.
0 0
0 0
Max cumulant level is set 

0 0
0 0
0 0
0 0
Max cumulant level is set to 2.
0 0
0 0
Max cumulant level is set to 2.
0 0
0 0
Max cumulant level is set to 2.
0 0
0 0
Max cumulant level is set to 2.
0 0
0 0
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 2.
0 0
0 0
Max cumulant level is set to 2.
Max cumulant level is set to 2.
0 0
Max cumulant level is set to 2.
Max cumulant level is set to 2.
0 0
Max cumulant level is set to 2.
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 2.
0 0
0 0
Max cumulant level is set to 2.
0 0
0 0
Max cumulant level is set to 2.
0 0
0 0
Max cumulant level is set to 2.
0 0
0 0
Max cumulant level is set to 1.
Max cumulant level is set to 1.
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
Max cumulant level is set to 2.
0 0
0 0
Max cumulant level is set to 2.
0 0
0 0
Max cumulant level is set to 2.
0 0
0 0
Max cumulant level is set to 2.
0 0
0 0
Max cumulant level is set to 1.
Max cumulant level is set to 1.
0 0
0 0


0 0
Max cumulant level is set to 2.
0 0
0 0
Max cumulant level is set to 2.
0 0
0 0
Max cumulant level is set to 2.
0 0
0 0
Max cumulant level is set to 1.
Max cumulant level is set to 1.
0 0
0 0
0 0
0 0
8 8
9 9
17 17
18 18
in func 18
Max cumulant level is set to 2.
0 0
0 0
Max cumulant level is set to 2.
0 0
0 0
Max cumulant level is set to 2.
0 0
0 0
Max cumulant level is set to 2.
0 0
0 0
Max cumulant level is set to 1.
Max cumulant level is set to 1.
0 0
0 0
0 0
0 0
8 8
9 9
17 17
18 18
in func 18
Max cumulant level is set to 2.
0 0
0 0
Max cumulant level is set to 2.
0 0
0 0
Max cumulant level is set to 2.
0 0
0 0
Max cumulant level is set to 2.
0 0
0 0
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 2.
Max cumulant level is set to 2.
0 0
0 0
Max cumulant level is set to 2.
Max cumulant level is set to 2.
0 0
Max cumulant level is set to 2.
Max cumulant level is set to 2.
0 0
Max cumulant level is set to 1.
Max cumulant level is set to 1

Max cumulant level is set to 2.
0 0
0 0
Max cumulant level is set to 2.
0 0
0 0
Max cumulant level is set to 2.
0 0
0 0
Max cumulant level is set to 2.
0 0
0 0
Max cumulant level is set to 1.
Max cumulant level is set to 1.
0 0
0 0
0 0
0 0
8 8
9 9
17 17
18 18
in func 18
Max cumulant level is set to 2.
0 0
0 0
Max cumulant level is set to 2.
0 0
0 0
Max cumulant level is set to 2.
0 0
0 0
Max cumulant level is set to 2.
0 0
0 0
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 2.
Max cumulant level is set to 2.
0 0
0 0
Max cumulant level is set to 2.
Max cumulant level is set to 2.
0 0
Max cumulant level is set to 2.
Max cumulant level is set to 2.
0 0
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 2.
Max cumulant level is set to 2.
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumulant level is set to 1.
Max cumul

[1 H^{ v_{1} }_{ v_{0} } T^{ c_{0} }_{ v_{0} } T^{ c_{0} }_{ v_{1} } a^{  }_{  },
 1/2 H^{ v_{1} }_{ v_{0} } T^{ c_{0} c_{1} }_{ v_{0} v_{2} } T^{ c_{0} c_{1} }_{ v_{1} v_{2} } a^{  }_{  },
 1/2 H^{ v_{1} }_{ v_{0} } T^{ c_{0} c_{1} }_{ v_{0} a_{0} } T^{ c_{0} c_{1} }_{ v_{1} a_{0} } a^{  }_{  },
 -1 H^{ c_{1} }_{ c_{0} } T^{ c_{0} }_{ v_{0} } T^{ c_{1} }_{ v_{0} } a^{  }_{  },
 -1 H^{ c_{1} }_{ c_{0} } T^{ a_{0} }_{ c_{0} } T^{ a_{0} }_{ c_{1} } a^{  }_{  },
 -1/2 H^{ c_{1} }_{ c_{0} } T^{ c_{0} c_{2} }_{ v_{0} v_{1} } T^{ c_{1} c_{2} }_{ v_{0} v_{1} } a^{  }_{  },
 -1 H^{ c_{1} }_{ c_{0} } T^{ c_{0} c_{2} }_{ v_{0} a_{0} } T^{ c_{1} c_{2} }_{ v_{0} a_{0} } a^{  }_{  },
 -1/2 H^{ c_{1} }_{ c_{0} } T^{ a_{0} a_{1} }_{ c_{0} c_{2} } T^{ a_{0} a_{1} }_{ c_{1} c_{2} } a^{  }_{  },
 1 H^{ a_{1} }_{ a_{0} } T^{ a_{0} }_{ c_{0} } T^{ a_{1} }_{ c_{0} } a^{  }_{  },
 1/2 H^{ a_{1} }_{ a_{0} } T^{ c_{0} c_{1} }_{ v_{0} a_{0} } T^{ c_{0} c_{1} }_{ v_{0} a_{1} } a^{  }_{  },
 1/2 H^{ a_{1} }_{ a_

In [17]:
dyall = combine_terms([Term.from_term(term) for term in HA + H0AA])
print_terms_ambit(dyall)

C0 += 2.0 * H1["c0,v0"] * T1["c0,v0"];
C0 += 2.0 * H1["a0,c0"] * T1["c0,a0"];
C0 += (1.0 / 2.0) * H2["c0,c1,v0,v1"] * T2["c0,c1,v0,v1"];
C0 += 1.0 * H2["c0,c1,v0,a0"] * T2["c0,c1,v0,a0"];
C0 += (1.0 / 2.0) * H2["a0,a1,c0,c1"] * T2["c0,c1,a0,a1"];
C0 += 1.0 * H1["v1,v0"] * T1["c0,v0"] * T1["c0,v1"];
C0 += (1.0 / 2.0) * H1["v1,v0"] * T2["c0,c1,v0,v2"] * T2["c0,c1,v1,v2"];
C0 += (1.0 / 2.0) * H1["v1,v0"] * T2["c0,c1,v0,a0"] * T2["c0,c1,v1,a0"];
C0 += -1.0 * H1["c1,c0"] * T1["c0,v0"] * T1["c1,v0"];
C0 += -1.0 * H1["c1,c0"] * T1["c0,a0"] * T1["c1,a0"];
C0 += (-1.0 / 2.0) * H1["c1,c0"] * T2["c0,c2,v0,v1"] * T2["c1,c2,v0,v1"];
C0 += -1.0 * H1["c1,c0"] * T2["c0,c2,v0,a0"] * T2["c1,c2,v0,a0"];
C0 += (-1.0 / 2.0) * H1["c1,c0"] * T2["c0,c2,a0,a1"] * T2["c1,c2,a0,a1"];
C0 += 2.0 * H1["a0,v0"] * T1["a1,v0"] * L1["a1,a0"];
C0 += 1.0 * H1["a0,v0"] * T2["a2,a3,v0,a1"] * L2["a2,a3,a0,a1"];
C0 += -2.0 * H1["a0,c0"] * T1["c0,a1"] * L1["a1,a0"];
C0 += -1.0 * H1["a0,c0"] * T2["c0,a1,a2,a3"] * L2["a2,a3,a0,

In [18]:
dyall_lambda = []

for term in dyall:
    for i in term.gradient_t():
        dyall_lambda.append(i)

dyall_lambda = combine_terms([i for i in dyall_lambda])
print_terms_ambit(dyall_lambda)

C1["c0,v0"] += 2.0 * H1["c0,v0"];
C1["c0,v0"] += 2.0 * H1["v1,v0"] * T1["c0,v1"];
C1["c0,v0"] += -2.0 * H1["c1,c0"] * T1["c1,v0"];
C1["c0,v0"] += -1.0 * H1["a1,a0"] * T2["c0,a0,v0,a2"] * L1["a2,a1"];
C1["c0,v0"] += 1.0 * H1["a1,a0"] * T2["c0,a2,v0,a0"] * L1["a2,a1"];
C1["c0,v0"] += (1.0 / 2.0) * H2["a2,a3,a0,a1"] * T2["c0,a0,v0,a4"] * L2["a2,a3,a1,a4"];
C1["c0,v0"] += (-1.0 / 2.0) * H2["a2,a3,a0,a1"] * T2["c0,a4,v0,a0"] * L2["a2,a3,a1,a4"];

C1["a0,v0"] += 2.0 * H1["a1,v0"] * L1["a1,a0"];
C1["a0,v0"] += 1.0 * H2["a2,a3,v0,a1"] * L2["a2,a3,a0,a1"];
C1["a0,v0"] += 2.0 * H1["v1,v0"] * T1["a1,v1"] * L1["a1,a0"];
C1["a0,v0"] += 1.0 * H1["v1,v0"] * T2["a2,a3,v1,a1"] * L2["a2,a3,a0,a1"];
C1["a0,v0"] += -1.0 * H1["a1,a0"] * T1["a2,v0"] * L1["a2,a1"];
C1["a0,v0"] += (-1.0 / 2.0) * H1["a1,a0"] * T2["a3,a4,v0,a2"] * L2["a3,a4,a1,a2"];
C1["a0,v0"] += -1.0 * H1["a2,a1"] * T1["a1,v0"] * L1["a2,a0"];
C1["a0,v0"] += -1.0 * H1["a2,a1"] * T2["a1,a4,v0,a3"] * L2["a2,a4,a0,a3"];
C1["a0,v0"] += (1.0 / 2.0)